# Intro to Torch Tensors

Why do ML people call multi-dimensional arrays "tensors"?  I have no idea.
If you do, please let me know.

In this notebook we demonstrate one of the core abstractions, the `torch.tensor`.
A `torch.tensor` is very similar to a `numpy.ndarray`. In fact there are efficient
conversions between the two, and the way to operate on them is very similar.

So why bother?

Because `torch.tensor` allows to:

  - execute operations on a GPU. This often speeds up applications
  tremendously.
  - track the partial derivatives/gradients of all operations at runtime, which is important
  to ML applications.

We will look at the latter into more detail in the "Intro do Autograd" section. Her we will
focus on creation of `torch.tensor` objects and how to operate on them.

[Reference Documentation](https://pytorch.org/docs/stable/tensors.html)


In [ ]:
import numpy as np
import torch
import math


## Various ways to create tensors


If you have no data and just want to create a `torch.tensor` object to be filled in later,
the most efficient way is to create an empty tensor.  This only allocates memory
and has no overhead otherwise.


In [ ]:
a = torch.empty((3, 4))
print(a)
print(a.shape)

If you are familiar with `numpy` this will all look familiar.


In [ ]:
b = torch.zeros((3, 4))
print(b)
c = torch.ones((3, 4))
print(c)
d = torch.rand((3, 4))
print(d)
print(d.type(), d.dtype)

The default element type is `torch.float32`. You can create tensors with other types.

Note that the element type is printed if it was explicitely specified.

In [ ]:
e = torch.ones((3, 4), dtype=torch.int32)
print(e)
print(e.type(), e.dtype)


Tensors can be converted to other types using the `.to()` method.  This creates a copy.

Check the [Reference Documentation](https://pytorch.org/docs/stable/tensors.html)
for available types.

In [ ]:
f = e.to(torch.uint8)
print(f)
print(id(f), f.type(), f.dtype)
print(id(e), e.type(), e.dtype)

We can also create tensors from python iterables.  This always creates a copy of the data
because the underlying representation is very different.


In [ ]:
g = torch.tensor([[1, 2, 3], (4, 5, 6)])
print(g)
print(g.type(), g.dtype)

Note that torch used the largest native integer type. Unlimited python integers
are not supported.

This only happens if all elements are integers. Otherwise torch goes back to the default.
But you can of course specify the desired `dtype`.


In [ ]:
h = torch.tensor([[1, 2, 3.14], (4, 5, 6)])
print(h)
print(h.type(), h.dtype)
i = torch.tensor([[1, 2, 3.14], (4, 5, 6)], dtype=torch.int32)
print(i)
print(i.type(), i.dtype)


This also works with numpy arrays.

In [ ]:
npa = np.ones((3, 4))
j = torch.tensor(npa)
print(j)
print(j.type(), j.dtype)

Note the `dtype`! This is a common gotcha.  Make sure you specify `dtype` when creating numpy
arrays to avoid trouble down the line.

This also created a copy of the data.

In [ ]:
j[0, 0] = 2
print(j)
print(npa)

Copying the data is often undesirable.  Torch offers methods to avoid this.
This works in both directions.  Note that no copy is made and operations
propagate all the way back.

In [ ]:
npb = np.ones((3, 4), dtype=np.float32)
k = torch.from_numpy(npb)
k[0, 0] = 2
print(k)
print(npb)

In [ ]:
npc = k.numpy()
print(npc)
npc[0, 1] = 2
print(npc)
print(k)
print(npb)

Slicing and broadcasting pretty much works as in `numpy`.
We are not going into the details here.

As expected, slices are views.

In [ ]:
l = k[:, 1:3]
l[0, 0] = 3.14
print(l)
print(k)

## Operations on Tensors

Pretty much like in `numpy` you can perform operations on `torch.tensor` objects.

  - per element scalar operations
  - per element operations on two tensors
  - linear algebra

By default, these create copies of the data. But there are in-place variants.

In [ ]:
m = torch.ones((3, 4))
n = m * 3
print(n)
print(m)

In-place variant, not creating a copy. Still returns a reference to the result.


In [ ]:
o = torch.ones((3, 4))
p = o.mul_(3)
print(p)
print(o)


All the usual suspects are available.


In [ ]:
q = torch.rand((20, ))
r = torch.sin(q)
print(r)
print(q)

... with their in-place variants

In [ ]:
s = q.sin_()
print(s)
print(q)


Some simple linear algebra. Here things can get confusing...


In [ ]:
t = torch.ones((3, 3))
u = torch.eye(3) * 7
v = t.matmul(u)
print(t)
print(u)
print(v)


... same in-place? This does not work!!

In [ ]:
print(t)
v = torch.matmul(t, u, out=t)
print(t)
print(u)
print(v)

This does work. Note that `x` and `y` refer to the same object.

In [ ]:
w = torch.ones((3, 3))
x = torch.zeros((3, 3))
y = torch.matmul(w, u, out=x)
print(w)
print(id(x), x)
print(id(y), y)

